In [1]:
from pyspark import SparkContext 

In [51]:
sc = SparkContext(master='local',appName='Olimpic_games')

In [21]:
%ls

deporte.csv      deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv           paises.csv
deportista.csv   juegos.csv           resultados.csv


In [52]:
path = '/home/cesar/Documents/prac_sp/curso-apache-spark-platzi/files/'
RDD_paises = sc.textFile(path+'paises.csv').map(lambda file : file.split(',') )

In [53]:
RDD_paises.take(20)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL'],
 ['15', 'Algeria', 'ALG'],
 ['16', 'Ali-Baba II', 'SWE'],
 ['17', 'Ali-Baba IV', 'SUI'],
 ['18', 'Ali-Baba IX', 'SUI'],
 ['19', 'Ali-Baba VI', 'SUI']]

# Count

Contar cuantos equipos hay, por sus siglas, para eso uso la función count(), pero antes, seleciono solo los valores distintos

In [58]:
RDD_paises.map(lambda x : x[2]).distinct().count()

231

En el caso de tener una cantidad de datos muy grande, y necesito saber cuantas rows hay, puedo usar la funcion de countApprox(), la cúal yo le doy una cantidad de milesegundos que quiero destinar a la tarea, en caso de que el timepo se agote y no haya terminado de contar, me devuleve el número de rows que llevaba para ese momento 

In [67]:
#El valor de countApprox esta en milesegundos 
RDD_paises.map(lambda x : x[2]).distinct().countApprox(20)

231

# Group BY

Contar la cantidad de veces que se repite una sigla.
Para esto tengo que ahcer un groupby. Primero mapeo las listas, tomo primero el valor que quiero colocar como key,
hago un groupByKey(), luego vuelvo a mapear, pero ahora mapeo los datos agrupados,y por último tomo 5 ejemplos con take()

In [59]:
RDD_paises.map(lambda x : (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [61]:
#As'i luce por dentro el mapValues()
RDD_paises.map(lambda x : (x[2],x[1])).groupByKey().mapValues(list).take(2)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise'])]

Filtar por algún país

# Filter 

In [66]:
RDD_paises.filter(lambda x : x[2] == 'MEX').take(5)

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX']]

# Union 

In [70]:
%ls

deporte.csv      deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv           paises.csv
deportista.csv   juegos.csv           resultados.csv


In [71]:
deportista1_rdd = sc.textFile(path+'deportista.csv').map(lambda x : x.split(','))
deportista2_rdd = sc.textFile(path+'deportista2.csv').map(lambda x : x.split(','))

deportista_rdd = deportista1_rdd.union(deportista2_rdd)

In [75]:
deportista_rdd.take(4)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273']]

In [77]:
deportista_rdd.count()

135572

Tomar una muestra aleatoria 

In [76]:
deportista_rdd.takeSample(False,10,seed=2)
#takeSample(Valores repetidos,cúatos registros quiero, semilla aleatoria)

[['134227', 'Carlos Erick Zegarra Presser', '1', '23', '202', '165', '794'],
 ['13661', 'Zenon Yanovich Bortkevich', '1', '27', '179', '80', '978'],
 ['106433', 'Massimo Scali', '1', '22', '172', '56', '509'],
 ['46508', 'Mohamed Hassanein', '1', '0', '0', '0', '308'],
 ['125686', 'Sabina Veit', '2', '22', '167', '59', '944'],
 ['112190', 'Austin Smith', '1', '23', '179', '73', '967'],
 ['29132', 'Eleni Doika', '2', '16', '168', '49', '419'],
 ['56472', 'Kaido Kaaberma', '1', '23', '194', '84', '331'],
 ['115105', 'Christopher Hugh Stevenson', '1', '27', '193', '91', '1096'],
 ['114685', 'Stanisaw Stefaski', '1', '25', '175', '62', '810']]

## Hacer un Join

In [78]:
RDD_paises.top(1)

[['id', 'equipo', 'sigla']]

In [80]:
deportista_rdd.top(1)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id']]

In [109]:
deportista_rdd.map(lambda x : [x[-1],x[:-1]]) \
                    .join(RDD_paises.map(lambda x: [x[0],x[2]])).takeSample(False,6,25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [131]:
resultado = sc.textFile(path+'resultados.csv').map(lambda x : x.split(','))
resultado.count()

271117

In [132]:
resultado.top(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99999', 'NA', '50604', '51', '568']]

Foltrar por los deortista que tengan medallas

In [133]:
resultado = resultado.filter(lambda x:'NA' not in x[1] )

In [134]:
resultado.top(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99993', 'Silver', '50600', '39', '207']]

Unir el rdd de deportista , equipo y de resultado  

In [121]:
deportista_equipo = (deportista_rdd.map(lambda x : [x[-1],x[:-1]]) 
                    .join(RDD_paises.map(lambda x: [x[0],x[2]]))
                    .map(lambda x : ( x[1][0][0] , (x[1][0][1:],x[1][1]) )  ) #Coloco el id del deportista como principal columna
)

In [122]:
deportista_equipo.top(2)

[('99999', (['Alexander Grant Alick Rennie', '1', '32', '182', '71'], 'RSA')),
 ('99998', (['Robert John Bob Renney', '1', '21', '178', '90'], 'AUS'))]

Por ultimo agrego la columna de los resultados

In [137]:
deportista_equipo.map(lambda x : [x[0],x[1]]).join(resultado.map(lambda x: [x[2],(x[0],x[1],x[3],x[4])])).take(2)

[('7597',
  ((['Bao Yingying', '2', '24', '172', '67'], 'CHN'),
   ('14078', 'Silver', '47', '445'))),
 ('17282',
  ((['Cai Huijue', '2', '16', '174', '63'], 'CHN'),
   ('33514', 'Bronze', '41', '171')))]

In [136]:
sc

<SparkContext master=local appName=Olimpic_games>

# Parar

In [138]:
sc.stop()